This notebook contains the script used to produce the charts and numbers from the events analysis.

In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)
sys.path.append(f'{project_path}/ft_events/src/utils')

from google.cloud import bigquery
from google.cloud import storage

import importlib

import numpy as np
import pandas as pd
from plotly import graph_objs as go
import seaborn as sns
import geopandas as gpd

import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.express as px
import ipywidgets as widgets

from fintrans_toolbox.src import table_utils as t
from fintrans_toolbox.src import bq_utils as bq



client = bigquery.Client()


## (1) Swiftonomics

### Taylor Swift concert dates:

<b> June 2024

Edinburgh (Murrayfield, EH12 5PJ, 70,000) x3 (7 (d),8+9 (e))

Liverpool (Anfield, L4 0TH, 61,276) x3 (13d,14d,15 e)

Cardiff (Principality,  CF10 1NS, 74,500) x1

London (Wembley, HA9 0WS, 90,000) x3

<b> August 2024

London (Wembley, HA9 0WS, 90,000) x3 

In [ ]:
def add_ts_periods(fig):

    # define covid lockdown start and end dates
    concert_periods = [
        ("2024-05-20", "2024-06-10"),
    ]
    # change colour of the covid lockdowns
    fillcolor = "grey"

    # add a dummy trace so that we can add the covid periods to the legend
    # add markers to square to make it show as a square in the legend.
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(color=fillcolor, symbol="square"),
            name="TS Concert",
        )
    )

    # add covid periods to plot
    for start_date, end_date in concert_periods:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=0.5,
            layer="below",
            line_width=0,
        )

    return fig

In [ ]:
def add_extra_wembley(fig):

    # define covid lockdown start and end dates
    concert_periods = [
        ("2024-07-20", "2024-08-10"),
    ]
    # change colour of the period
    fillcolor = "gainsboro"

    # add a dummy trace so that we can add the covid periods to the legend
    # add markers to square to make it show as a square in the legend.
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(color=fillcolor, symbol="square"),
            name="Extra Wembley (HA)",
        )
    )

    # add covid periods to plot
    for start_date, end_date in concert_periods:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=0.5,
            layer="below",
            line_width=0,
        )

    return fig

In [ ]:
def calc_index_yoy(df, need_date_cols, group_list):
    
    result = df.copy()
    if need_date_cols is True:
        result['year'] = result['date_time'].dt.year
        result['month'] = result['date_time'].dt.month
    
    metrics = ['spend', 'transactions', 'cardholders']
    month_group = group_list + ['month']

    for i in metrics:
        # calc year-on-year differences
        result[f'yoy_{i}'] = result.groupby(month_group)[f'{i}'].diff(periods=1)

        # calc year-on-year % change
        result[f'yoy_{i}_perc'] = result.groupby(month_group)[f'{i}'].pct_change(periods=1)*100

        # index to 2019 average
        result[f'index_{i}_2019'] = result.groupby(group_list)[f'{i}'].transform(lambda x: x / (x.iloc[0:11].mean(axis = 0)))
        
        # index to jan 2019
        result[f"index_{i}"] = result.groupby(group_list)[f"{i}"].transform(
        lambda x: x / x.iloc[0]
    )
        
    return result

In [ ]:
def plot_metrics(df, postal_level, metric, event):
    if 'yoy' in metric:
        if 'perc' in metric:
            perc = 'percentage change'
        else:
            perc = ''
        prefix = f'Year on year {perc}'
        suffix = metric.split('_')[1]
        y_label = f'{prefix} sum {suffix}'
    if 'index' in metric:
        prefix = 'Index'
        suffix = metric.split('_')[1]
        y_label = f'{prefix} sum {suffix}'
    
    if event == 'TS':
        codes = ts_all_codes
        
    if event == 'SN':
        codes = sn_all_codes
    
    if event not in ('SN', 'TS'):
        raise ValueError(f"Argument event must be one of 'TS' for Taylor Swift events or 'SN' for Six Nations event")
    
    
    venue_df = df[(df['merchant_location_level'] == postal_level) & (df['merchant_location'].isin(codes))].groupby(['date_time', 'merchant_location']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index()
    uk_df = df[(df['merchant_location_level'] == postal_level) & (~df['merchant_location'].isin(codes)) & (df['merchant_location'] != 'UNKNOWN')].groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index()
    uk_df['merchant_location'] = 'UK'
    
    full_ts_df = pd.concat([venue_df, uk_df])
    full_ts_df = calc_index_yoy(full_ts_df, need_date_cols = True, group_list = ['merchant_location'])
    
    level = postal_level[7:].lower()
    
    fig = px.line(
    full_ts_df,

    x="date_time",
    y=metric,
    color = 'merchant_location',
    title=f"{prefix} {suffix} at stadium postal {level}",
    height = 500,
    template = 'simple_white',
    color_discrete_map=all_areas_uk_colours

    )
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))

    fig.update_layout(legend_title_text=f'Venue postal {level}:', 
                      title_subtitle_text=f'MCG = {pick_mcg}',
                      yaxis_title = f'{prefix} sum {suffix}',
                     xaxis_title = 'Date')
    fig.update_traces(selector=dict(name='UK'), line=dict(dash='dash'))  #dashed line for UK
    
    if event == 'TS':
        fig = add_ts_periods(fig)
        fig = add_extra_wembley(fig)
        
    if event == 'SN':
        fig = add_nations_periods(fig)

    fig.show()
    
    return full_ts_df

In [ ]:
def location_level_validation(df, merch_card):
    
    # Counts number of numerals in each location code
    df["number"] = df[f'{merch_card}'].str.extract(
        "(\d+)", expand=False
    )

    if len(df[f'{merch_card.split("_")[0]}_location_level'].unique()) >1:

        first_df = df[df[f'{merch_card.split("_")[0]}_location_level'] == 'POSTAL_AREA'].copy()
        second_df = df[df[f'{merch_card.split("_")[0]}_location_level'] == 'POSTAL_DISTRICT'].copy()

        # no nums in postal area
        first_df = first_df.loc[first_df["number"].isna() == True].drop(
            "number", axis=1
        )

        # nums in postal district
        second_df = second_df.loc[second_df["number"].isna() == False].drop(
            "number", axis=1
        )


        clean_df = pd.concat([first_df, second_df])

    else:
        if df[f'{merch_card.split("_")[0]}_location_level'][1] == 'POSTAL_AREA':

            clean_df = df.loc[df["number"].isna() == True].drop(
                "number", axis=1
            )

        if df[f'{merch_card.split("_")[0]}_location_level'][1] == 'POSTAL_DISTRICT':

            clean_df = df.loc[df["number"].isna() == False].drop(
                "number", axis=1
            )

    return clean_df

In [ ]:
ts_pas = ['EH', 'L', 'CF', 'HA']
ts_pds = ('EH12', 'L4', 'CF10', 'HA9')

ts_all_codes = ['EH', 'L', 'CF', 'HA', 'EH12', 'L4', 'CF10', 'HA9']

areas_uk_colours = {'UK': '#003c57',
 'CF': '#a8bd3a',
 'EH': '#27a0cc',
 'HA': '#0f8243',
 'L': '#F46A25'}

all_areas_uk_colours = {'UK': '#003c57',
 'CF': '#a8bd3a',
 'EH': '#27a0cc',
 'HA': '#0f8243',
 'L': '#F46A25',
'CF10': '#a8bd3a',
 'EH12': '#27a0cc',
 'HA9': '#0f8243',
 'L4': '#F46A25',
                       'CF': '#a8bd3a',
 'EH': '#27a0cc',
 'TW': '#F46A25',
                       'CF10': '#a8bd3a',
 'EH12': '#27a0cc',
 'TW2': '#F46A25'}

In [ ]:
def calc_index_yymm(df, group, need_datetime):
    
    if need_datetime is True:
        df['year'] = df['date_time'].dt.year
        df['month'] = df['date_time'].dt.month
    
    month_group = group + ['month']
    
    df = df.sort_values(['date_time']).reset_index(drop=True)
    # index spend
    df["index_spend"] = df.groupby(group)["spend"].transform(
        lambda x: x / x.iloc[0]
    )
    # index cardholders
    df["index_cardholders"] = df.groupby(group)["cardholders"].transform(
        lambda x: x / x.iloc[0]
    )
    # month-on-month
    df['mm_perc_spend'] = df.groupby(group)['index_spend'].pct_change(periods=1)*100
    df['mm_perc_cardholders'] = df.groupby(group)['index_cardholders'].pct_change(periods=1)*100

    # year-on-year
    df['yy_perc_spend'] = df.groupby(month_group)['index_spend'].pct_change(periods=1)*100
    df['yy_perc_cardholders'] = df.groupby(month_group)['index_cardholders'].pct_change(periods=1)*100
    
    return df

In [ ]:
pick_mcg = 'All'

In [ ]:
sql_spend = f"""SELECT time_period_value, merchant_location_level,
merchant_location, cardholder_issuing_country, mcg, mcc, spend, transactions, cardholders
FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
WHERE time_period = 'Month' AND
mcg = '{pick_mcg}' AND
mcc = 'All' AND
merchant_location_level != 'All' AND
merchant_location_level != 'POSTAL_SECTOR' AND
cardholder_issuing_level != 'All' 
GROUP BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders
ORDER BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders"""

sml_df = client.query(sql_spend).to_dataframe()
ts_df = t.create_date_time(sml_df)
ts_df = location_level_validation(df = ts_df, merch_card = 'merchant_location')

In [ ]:
sql_spend = f"""SELECT time_period_value, merchant_location_level,
merchant_location, cardholder_issuing_country, mcg, mcc, spend, transactions, cardholders
FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
WHERE time_period = 'Month' AND
mcg = '{pick_mcg}' AND
mcc = 'All' AND
merchant_location_level != 'All' AND
merchant_location_level != 'POSTAL_SECTOR' AND
cardholder_issuing_level != 'All' 
GROUP BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders
ORDER BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders"""

sml_df = client.query(sql_spend).to_dataframe()
ts_df = t.create_date_time(sml_df)
ts_df = location_level_validation(df = ts_df, merch_card = 'merchant_location')

<b>options</b>:
'yoy_spend', 'yoy_spend_perc',
       'index_spend_2019', 'index_spend', 'yoy_transactions',
       'yoy_transactions_perc', 'index_transactions_2019',
       'index_transactions', 'yoy_cardholders', 'yoy_cardholders_perc',
       'index_cardholders_2019', 'index_cardholders'

In [ ]:
df = plot_metrics(df = ts_df, postal_level = 'POSTAL_DISTRICT', metric = 'index_spend', event = 'TS')

In [ ]:
df = plot_metrics(df = ts_df, postal_level = 'POSTAL_DISTRICT', metric = 'yoy_spend_perc', event = 'TS')

### Spend from UK cardholders outside postal location

In [ ]:
ts_out_sql = f"""SELECT time_period_value,
merchant_location, cardholder_location,  mcg, spend, transactions, cardholders
FROM ons-fintrans-data-prod.fintrans_visa.retail_performance_high_streets_towns
WHERE time_period = 'Month' AND
mcg = 'All' AND
merchant_location IN  {ts_pds} AND
cardholder_location_level = 'POSTAL_DISTRICT'
ORDER BY time_period_value,  merchant_location, cardholder_location, mcg, spend, transactions, cardholders"""

oc_df = client.query(ts_out_sql).to_dataframe()
oc_df = t.create_date_time(oc_df)


In [ ]:
# remove non-district locations

# remove codes with no numbers
oc_df["number"] = oc_df['cardholder_location'].str.extract(
        "(\d+)", expand=False
    )
oc_df = oc_df.loc[oc_df["number"].isna() == False].drop(
                "number", axis=1
            )

# make sure no spaces in code
oc_df["number"] = oc_df['cardholder_location'].str.count(" ")

if len(oc_df['number'].unique()) == 1:
    oc_df = oc_df.drop(
                "number", axis=1
            )
else:
    oc_df = oc_df.loc[oc_df["number"] > 1].drop(
                "number", axis=1
            )

#### Identify and exclude nearby districts

In [ ]:
from google.cloud import storage
from geopandas import gpd

# Loading shapes
client = storage.Client()
#bq.boundary_file_download(client, postal_level = "postcode_district", output_location = '')
district_shape = gpd.read_file('postcode_district.shp')
client = bigquery.Client()


In [ ]:
# saving geo shapes of arena districts
venues = district_shape[district_shape['postdistri'].isin(ts_pds)].geometry
# saving postal district name + geographies
venue_names = district_shape[district_shape['postdistri'].isin(ts_pds)]
# creating a 25km buffer around the venue geographies
buffer = venues.buffer(25000)

In [ ]:
# done individually for each district - since the buffer around EH does not need to be excluded for CF etc
# Exclude district geoetries that intersect the 25km buffer
buffered1 = district_shape[
                        (~district_shape.geometry.intersects(buffer.iloc[0]))].copy() # Exclude those within the set distance
buffered2 = district_shape[
                        (~district_shape.geometry.intersects(buffer.iloc[1]))].copy() 
buffered3 = district_shape[
                        (~district_shape.geometry.intersects(buffer.iloc[2]))].copy() 
buffered4 = district_shape[
                        (~district_shape.geometry.intersects(buffer.iloc[3]))].copy() 

In [ ]:
# Add a column to the buffered df geographies for the name of the district.
# venue_names and buffer have same order of districts
buffered1['merchant_location'] = venue_names['postdistri'].iloc[0]
buffered2['merchant_location'] = venue_names['postdistri'].iloc[1]
buffered3['merchant_location'] = venue_names['postdistri'].iloc[2]
buffered4['merchant_location'] = venue_names['postdistri'].iloc[3]

In [ ]:
# Saving unique postal districts for each of the buffered zones
cf10_buffered = buffered1['postdistri'].unique()
eh12_buffered = buffered2['postdistri'].unique()
ha9_buffered = buffered3['postdistri'].unique()
l4_buffered = buffered4['postdistri'].unique()

In [ ]:
# Filtering the original df (non-geo) to get df of the buffered cardholder locations for each venue
l4_df = oc_df[(oc_df['merchant_location'] == 'L4') & (oc_df['cardholder_location'].isin(l4_buffered))].groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()
l4_df['merchant_location'] = 'L4'
cf10_df = oc_df[(oc_df['merchant_location'] == 'CF10') & (oc_df['cardholder_location'].isin(cf10_buffered))].groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()
cf10_df['merchant_location'] = 'CF10'
eh12_df = oc_df[(oc_df['merchant_location'] == 'EH12') & (oc_df['cardholder_location'].isin(eh12_buffered))].groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()
eh12_df['merchant_location'] = 'EH12'
ha9_df = oc_df[(oc_df['merchant_location'] == 'HA9') & (oc_df['cardholder_location'].isin(ha9_buffered))].groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()
ha9_df['merchant_location'] = 'HA9'


In [ ]:
# Adding in international spend 
# Comment out if not needed

l4_int = ts_df[(ts_df['merchant_location_level'] == 'POSTAL_DISTRICT') & (ts_df['merchant_location'] == 'L4') & (~ts_df['cardholder_issuing_country'].isin(['All', 'UNITED KINGDOM']))].groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()
cf10_int = ts_df[(ts_df['merchant_location_level'] == 'POSTAL_DISTRICT') & (ts_df['merchant_location'] == 'CF10') & (~ts_df['cardholder_issuing_country'].isin(['All', 'UNITED KINGDOM']))].groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()
eh12_int = ts_df[(ts_df['merchant_location_level'] == 'POSTAL_DISTRICT') & (ts_df['merchant_location'] == 'EH12') & (~ts_df['cardholder_issuing_country'].isin(['All', 'UNITED KINGDOM']))].groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()
ha9_int = ts_df[(ts_df['merchant_location_level'] == 'POSTAL_DISTRICT') & (ts_df['merchant_location'] == 'HA9') & (~ts_df['cardholder_issuing_country'].isin(['All', 'UNITED KINGDOM']))].groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()

l4_df = pd.concat([l4_df, l4_int]).groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()
cf10_df = pd.concat([cf10_df, cf10_int]).groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()
eh12_df = pd.concat([eh12_df, eh12_int]).groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()
ha9_df = pd.concat([ha9_df, ha9_int]).groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index().copy()


l4_df['merchant_location'] = 'L4'
cf10_df['merchant_location'] = 'CF10'
eh12_df['merchant_location'] = 'EH12'
ha9_df['merchant_location'] = 'HA9'


In [ ]:
# Combining 'outie' venue spend dfs
summed_outies = pd.concat([l4_df, cf10_df, eh12_df, ha9_df])

In [ ]:
# Calculating index/yy
summed_outies = calc_index_yoy(summed_outies, need_date_cols = True, group_list = ['merchant_location'])
l4_df = calc_index_yoy(l4_df, need_date_cols = True, group_list = ['merchant_location'])
cf10_df = calc_index_yoy(cf10_df, need_date_cols = True, group_list = ['merchant_location'])
eh12_df = calc_index_yoy(eh12_df, need_date_cols = True, group_list = ['merchant_location'])
ha9_df = calc_index_yoy(ha9_df, need_date_cols = True, group_list = ['merchant_location'])

In [ ]:
ha9_df.columns

In [ ]:
for df in [summed_outies, l4_df, cf10_df, eh12_df, ha9_df]:
    
    if len(df['merchant_location'].unique()) >1:
        title_text = 'Summed venue index outer-spend'
        loc = 'NA'
    else:
        loc = df['merchant_location'][0]
        title_text = f'{loc} index outer-spend'
    
    fig = px.line(
    df,
    x="date_time",
    y = "index_spend",
    template='simple_white',
    color = 'merchant_location',
    height = 400,
    width = 800,
    title = title_text)
    
    fig = add_ts_periods(fig)
    
    if loc == 'HA9':
        fig = add_extra_wembley(fig)
    else:
        pass
    fig.show()

### International spend at districts

In [ ]:
client = bigquery.Client()

sql = f"""SELECT time_period_value, merchant_location, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'POSTAL_DISTRICT' AND
  cardholder_issuing_level = 'International' AND
  cardholder_issuing_country = 'All' AND
  mcg = 'All' AND
  merchant_location IN  {ts_pds}
  ORDER BY time_period_value, merchant_location
  """

df_full_int = client.query(sql).to_dataframe()
df_full_int = t.create_date_time(df_full_int)

In [ ]:
ts_int_spend = calc_index_yymm(df_full_int, group = ['merchant_location'], need_datetime = True)

In [ ]:
fig = px.line(
ts_int_spend,
x="date_time",
y = "index_spend",
template='simple_white',
color = 'merchant_location',
height = 500,
width = 1000,
title = 'Index international spend at concert districts')

fig = add_ts_periods(fig)

fig = add_extra_wembley(fig)

fig.show()

------------------------------

## (2) Six Nations

In [ ]:
def add_nations_periods(fig):

    # define covid lockdown start and end dates
    game_periods = [
        ("2019-02-01", "2019-03-16"),
        ("2020-02-01", "2020-03-08"),
        ("2021-02-06", "2021-03-26"),
        ("2022-02-01", "2022-03-01"),
        ("2023-02-01", "2023-03-01"),
        ("2024-02-01", "2024-03-01"),
        ("2025-02-01", "2025-03-15"),
    ]
    # change colour of the covid lockdowns
    fillcolor = "grey"

    # add a dummy trace so that we can add the covid periods to the legend
    # add markers to square to make it show as a square in the legend.
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(color=fillcolor, symbol="square"),
            name="Six Nations",
        )
    )

    # add covid periods to plot
    for start_date, end_date in game_periods:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=0.5,
            layer="below",
            line_width=0,
        )

    return fig

def plot_int_games(postal_level, metric):

    if 'AREA'  in postal_level:
        venues = six_nation_areas
        loc_level = 'area'

    else:
        venues = six_nation_districts
        loc_level = 'district'

    venue_international = sn_df[(sn_df['merchant_location'].isin(venues))& (sn_df['cardholder_issuing_country'].isin(six_nation_countries))].groupby(['date_time', 'merchant_location','cardholder_issuing_country', 'mcg',]).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index()
    uk_international = sn_df[(~sn_df['merchant_location'].isin(venues)) & (sn_df['cardholder_issuing_country'].isin(six_nation_countries))].groupby(['date_time','cardholder_issuing_country', 'mcg',]).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index()
    uk_international['merchant_location'] = 'UK'
    full_international = pd.concat([ venue_international, uk_international])
    
    full_international = calc_index_yoy(full_international, need_date_cols = True, group_list = ['merchant_location', 'cardholder_issuing_country'])
    
    if 'yoy' in metric:
            if 'perc' in metric:
                perc = 'percentage change in'
            else:
                perc = ''
            prefix = f'year on year {perc}'
            suffix = metric.split('_')[1]
            y_label = f'{prefix} sum {suffix}'
    if 'index' in metric:
        prefix = 'index'
        suffix = metric.split('_')[1]
        y_label = f'{prefix} sum {suffix}'

    for venue in venues:
        fig = px.line(
        full_international[ (full_international['merchant_location'] == venue)],
        x="date_time",
        y=metric,
        color = "cardholder_issuing_country",
        title=f"Six Nation country {prefix} {suffix} at {venue}",
        template = "simple_white",
        height = 500
        )
        fig = add_nations_periods(fig)

        fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))

        fig.update_layout(legend_title_text='Inbound spend country:', title_subtitle_text=f'MCG = {pick_mcg}')


        # Filter matches_log for the current area
        matches_in_area = matches_log[matches_log[f'{loc_level}'] == venue]

        for _, match in matches_in_area.iterrows():
            try:
                # Extract spend value for the specific date_time and cardholder_issuing_country
                spend_value = full_international[
                    (full_international['date_time'] == match['date_time']) & 
                    (full_international['cardholder_issuing_country'] == match['country']) & 
                    (full_international['merchant_location'] == match[f'{loc_level}'])
                ][f'{metric}'].iloc[0]

                # Adding vertical line at the match date
                fig.add_vline(
                    x=match['date_time'],
                    line_dash='dot',
                    line_color='gray',
                    opacity=0.6
                )

                # Adding text annotations at the corresponding 'spend' value
                fig.add_annotation(
                    x=match['date_time'],  # Position the annotation at the same x position
                    y=spend_value,  # Use the extracted spend value as the y position
                    text=match['host_name'] + ' v '+  match['game_name'][2:].title(),
                    font=dict(size=10, color="black",style="italic"),  # Font style of the annotation
                    align='center',  # Text alignment,
                )
            except IndexError: 
                pass

        # Show the plot for the current area
        fig.show()
        
    return full_international

In [ ]:
def plot_int_mcg_spend(df, metric, merchant_location, country):
    if len(merchant_location) <= 2:
        loc_level = 'POSTAL_AREA'
    if len(merchant_location) > 2:
        loc_level = 'POSTAL_DISTRICT'

    mcg_df = sn_mcg_df[(sn_mcg_df['merchant_location'] == merchant_location) & (sn_mcg_df['cardholder_issuing_country'] == country) & (sn_mcg_df['merchant_location_level'] == loc_level)]
    mcg_df = calc_index_yoy(mcg_df, need_date_cols = False, group_list = ['mcg'])
    
    fig = px.line(
    mcg_df,

    x="date_time",
    y=metric,
    color = 'mcg',
    title=f"{country} MCG spend at {merchant_location}",
    height = 500,
    template = 'simple_white',
    color_discrete_map=all_areas_uk_colours

    )

    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))



    fig.show()

In [ ]:
six_nation_areas = ['CF', 'EH', 'TW']
six_nation_districts = ['CF10', 'EH12', 'TW2']
six_nation_countries = ('FRANCE', 'ITALY', 'REPUBLIC OF IRELAND')
areas_uk_colours = {'UK': '#003c57',
 'CF': '#a8bd3a',
 'EH': '#27a0cc',
 'TW': '#F46A25'}

districts_uk_colours = {'UK': '#003c57',
 'CF10': '#a8bd3a',
 'EH12': '#27a0cc',
 'TW2': '#F46A25'}

sn_all_codes = ('CF', 'EH', 'TW', 'CF10', 'EH12', 'TW2')


matches_log = pd.DataFrame(
    # Cardiff
    [{'date_time':'2019-03-01', 'area': 'CF', 'district': 'CF10', 'country': 'REPUBLIC OF IRELAND'},
    {'date_time':'2020-02-01', 'area': 'CF', 'district': 'CF10',  'country': 'ITALY'},
    {'date_time':'2020-02-01', 'area': 'CF', 'district': 'CF10',   'country': 'FRANCE'},
    {'date_time':'2022-03-01', 'area': 'CF', 'district': 'CF10',   'country': 'FRANCE'},
    {'date_time':'2022-03-01', 'area': 'CF', 'district': 'CF10',   'country': 'ITALY'},
    {'date_time':'2023-02-01', 'area': 'CF', 'district': 'CF10',   'country': 'REPUBLIC OF IRELAND'},
    {'date_time':'2024-03-01', 'area': 'CF', 'district': 'CF10',   'country': 'FRANCE'},
    {'date_time':'2024-03-01', 'area': 'CF', 'district': 'CF10',   'country': 'ITALY'},
    {'date_time':'2025-02-01', 'area': 'CF', 'district': 'CF10',   'country': 'REPUBLIC OF IRELAND'},
     
    # Edinburgh
    {'date_time':'2019-02-01', 'area': 'EH', 'district': 'EH12',   'country': 'ITALY'},
    {'date_time':'2019-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'REPUBLIC OF IRELAND'},
    {'date_time':'2020-03-01', 'area': 'EH', 'district': 'EH12', 'country': 'FRANCE'},
    {'date_time':'2022-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'FRANCE'},
    {'date_time':'2023-03-01', 'area': 'EH', 'district': 'EH12', 'country': 'REPUBLIC OF IRELAND'},
    {'date_time':'2023-03-01', 'area': 'EH', 'district': 'EH12', 'country': 'ITALY'},
    {'date_time':'2024-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'FRANCE'},
    {'date_time':'2025-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'ITALY'},
    {'date_time':'2025-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'REPUBLIC OF IRELAND'},
    
    # Twickenham
    {'date_time':'2019-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'FRANCE'},
    {'date_time':'2019-03-01', 'area': 'TW', 'district': 'TW2', 'country': 'ITALY'},
    {'date_time':'2020-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'REPUBLIC OF IRELAND'},
    {'date_time':'2022-03-01', 'area': 'TW', 'district': 'TW2', 'country': 'REPUBLIC OF IRELAND'},
    {'date_time':'2023-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'ITALY'},
    {'date_time':'2023-03-01', 'area': 'TW', 'district': 'TW2', 'country': 'FRANCE'},
    {'date_time':'2024-03-01', 'area': 'TW', 'district': 'TW2', 'country': 'REPUBLIC OF IRELAND'},
    {'date_time':'2025-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'FRANCE'},
    {'date_time':'2025-03-01', 'area': 'TW', 'district': 'TW2', 'country': 'ITALY'},])


matches_log['game_name'] = 'v ' + matches_log['country']
area_to_host = {'CF': 'Wales', 'TW': 'England', 'EH': 'Scotland'}

matches_log['host_name'] = matches_log['area'].map(area_to_host)


uk_matches_log = pd.DataFrame(
    # Cardiff
    [{'date_time':'2019-02-01', 'area': 'CF', 'district': 'CF10', 'country': 'ENGLAND'},
     {'date_time':'2022-02-01', 'area': 'CF', 'district': 'CF10', 'country': 'SCOTLAND'},
     {'date_time':'2023-03-01', 'area': 'CF', 'district': 'CF10', 'country': 'ENGLAND'},
     {'date_time':'2024-02-01', 'area': 'CF', 'district': 'CF10', 'country': 'SCOTLAND'},
     {'date_time':'2025-03-01', 'area': 'CF', 'district': 'CF10', 'country': 'ENGLAND'},

    
    # Edinburgh
     {'date_time':'2019-03-01', 'area': 'EH', 'district': 'EH12', 'country': 'WALES'},
     {'date_time':'2020-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'ENGLAND'},
     {'date_time':'2022-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'ENGLAND'},
     {'date_time':'2023-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'WALES'},
     {'date_time':'2024-02-01', 'area': 'EH', 'district': 'EH12', 'country': 'ENGLAND'},
     {'date_time':'2025-03-01', 'area': 'EH', 'district': 'EH12', 'country': 'WALES'},
    
    # Twickenham
     {'date_time':'2019-03-01', 'area': 'TW',  'district': 'TW2','country': 'SCOTLAND'},
     {'date_time':'2020-03-01', 'area': 'TW', 'district': 'TW2', 'country': 'WALES'},
     {'date_time':'2022-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'WALES'},
     {'date_time':'2023-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'SCOTLAND'},
     {'date_time':'2024-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'WALES'},
    {'date_time':'2025-02-01', 'area': 'TW', 'district': 'TW2', 'country': 'SCOTLAND'}])


uk_matches_log['game_name'] = 'v ' + uk_matches_log['country']

area_to_host = {'CF': 'Wales', 'TW': 'England', 'EH': 'Scotland'}

uk_matches_log['host_name'] = uk_matches_log['area'].map(area_to_host)

matches_log['date_time'] = pd.to_datetime(matches_log['date_time'], format='%Y-%m-%d')
uk_matches_log['date_time'] = pd.to_datetime(uk_matches_log['date_time'], format='%Y-%m-%d')



## 2019-2025 Fixtures

* (F/M) = February/March
* 2021 the games were held without spectators

|  | 2019 |  | ----  |  | 2020 |  |   
|---------| ----------|---------|------------|---------|----------|---------|

| Cardiff | Edinburgh | London | ---- | Cardiff | Edinburgh | London |
|---------|----------|---------|------|---------|----------|---------|
| v England (F) | v Italy (F) | v France (F) |---- | v Italy (F) | v England (F) | v Ireland (F) |
| v Ireland (M) | v Ireland (F) | v Italy (M) |---- | v France (F) | v France (M) | v Wales (M) |
|   | v Wales (M) | v Scotland (M) |----  | | | |



|  | 2022 |  | ----  |  | 2023 |  |  
|---------|----------|---------|------|---------|----------|---------|

| Cardiff | Edinburgh | London | ---- | Cardiff | Edinburgh | London |
|---------|----------|---------|--------|---------|----------|---------|
| v Scotland (F) | v England (F) | v Wales (F) | ----  | v Ireland (F) | v Wales (F) | v Scotland (F) |
| v France (M) | v France (F) | v Ireland (M) |----  | v England (M) | v Ireland (M) | v Italy (F) |
| v Italy (M) |  |   |---- |   | v Italy (M) | v France (M) |


|  | 2024 |  |  ---- | | 2025 |  |
|---------|----------|---------|------|---------|----------|---------|

| Cardiff | Edinburgh | London | ---- | Cardiff | Edinburgh | London |
|---------|----------|---------|--------|---------|----------|---------|
| v Scotland (F) | v England (F) | v Wales (F) | ----  | v  Ireland (F) | v Italy (F) | v  France (F) |
| v France (M) | v France (F) | v Ireland (M) | ----  | v England (M) | v Ireland (F) | v  Scotland (F) |
| v Italy (M) |  |   |---- |   | v Wales (M) | v  Italy (M) |


In [ ]:
sql_spend = f"""SELECT time_period_value, merchant_location_level,
merchant_location, cardholder_issuing_country, mcg, mcc, spend, transactions, cardholders
FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
WHERE time_period = 'Month' AND
mcg = '{pick_mcg}' AND
mcc = 'All' AND
merchant_location_level != 'All' AND
cardholder_issuing_level != 'All' AND
merchant_location_level != 'POSTAL_SECTOR'
GROUP BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders
ORDER BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders"""

sn_df = client.query(sql_spend).to_dataframe()
sn_df = t.create_date_time(sn_df)
sn_df = location_level_validation(df = sn_df, merch_card = 'merchant_location')

<b>options</b>:
'yoy_spend', 'yoy_spend_perc',
       'index_spend_2019', 'index_spend', 'yoy_transactions',
       'yoy_transactions_perc', 'index_transactions_2019',
       'index_transactions', 'yoy_cardholders', 'yoy_cardholders_perc',
       'index_cardholders_2019', 'index_cardholders'

In [ ]:
plot_metrics(df = sn_df, postal_level = 'POSTAL_DISTRICT', metric = 'yoy_spend_perc', event = 'SN')

#### International visiting country spend at UK venues

In [ ]:
plot_int_games(postal_level = 'POSTAL_DISTRICT', metric = 'index_spend')

#### International cardholder MCG spend

In [ ]:
mcgs = ('QSR', 'RESTAURANTS', 'RETAIL GOODS', 'LODGING', 'FOOD & GROCERY')
#mcg IN {mcgs}


In [ ]:
sql_spend_mcg = f"""SELECT time_period_value, merchant_location_level,
merchant_location, cardholder_issuing_country, mcg, mcc, spend, transactions, cardholders
FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
WHERE time_period = 'Month' AND
mcc = 'All' AND
merchant_location_level != 'All' AND
cardholder_issuing_country IN {six_nation_countries} AND
merchant_location IN {sn_all_codes} AND
mcg IN {mcgs}
GROUP BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders
ORDER BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders"""

sn_mcg_df = client.query(sql_spend_mcg).to_dataframe()
sn_mcg_df = t.create_date_time(sn_mcg_df)

In [ ]:
sn_mcg_df = location_level_validation(df = sn_mcg_df, merch_card = 'merchant_location')

'yoy_spend',
       'yoy_spend_perc', 'index_spend_2019', 'index_spend', 'yoy_transactions',
       'yoy_transactions_perc', 'index_transactions_2019',
       'index_transactions', 'yoy_cardholders', 'yoy_cardholders_perc',
       'index_cardholders_2019', 'index_cardholders'

In [ ]:
plot_int_mcg_spend(df = sn_mcg_df, metric = 'index_spend',merchant_location = 'CF10', country = 'FRANCE')

------------------------------

## (3) Holyhead port

In [ ]:
treated_city = 'Holyhead' 
holyhead_districts = ['LL33'] + [ f'LL{i}' for i in range(54, 79)]
treatment_period = "202411" # Month of event
data_from = 'All' # Can be All, Domestic or International
event_dates = ['202411']
first_month = '202111'


# Weekly ship indicators from https://www.ons.gov.uk/economy/economicoutputandproductivity/output/datasets/weeklyshippingindicators
# the below values are calculated by subtracting the cargo ship visit monthly values from total ship monthly visits

ship_data = {
    'date_time': [
    '11/01/2021', '12/01/2021', '01/01/2022', '02/01/2022', '03/01/2022', '04/01/2022', '05/01/2022', '06/01/2022', '07/01/2022', '08/01/2022',
    '09/01/2022', '10/01/2022', '11/01/2022', '12/01/2022', '01/01/2023', '02/01/2023', '03/01/2023', '04/01/2023', '05/01/2023', '06/01/2023',
    '07/01/2023', '08/01/2023', '09/01/2023', '10/01/2023', '11/01/2023', '12/01/2023', '01/01/2024', '02/01/2024', '03/01/2024', '04/01/2024',
    '05/01/2024', '06/01/2024', '07/01/2024', '08/01/2024', '09/01/2024', '10/01/2024', '11/01/2024', '12/01/2024', '01/01/2025', '02/01/2025',
    '03/01/2025'],
    
    'ships': [
    197, 213, 211, 216, 245, 267, 272, 274, 284, 291, 267, 253, 224, 217, 198, 191, 250, 264, 295, 279, 294, 294, 265, 272, 217, 219, 172, 191,
    195, 222, 260, 284, 306, 285, 272, 225, 207, 48, 101, 197, 176]
}

# Create the DataFrame
ship_df = pd.DataFrame(ship_data)

# Convert the 'date_time' column to datetime objects
ship_df['date_time'] = pd.to_datetime(ship_df['date_time'], format='%m/%d/%Y')



holyhead_colours = {'LL65' : '#206095', # ocean blue
                    'Wider Anglesey': '#A8BD3A', # spring green
                    'Rest of UK' : '#871A5B', # beetroot purple
                    'Rest of International' : 'dimgrey', 
                    'Northern Ireland': '#746CB1', # lavender purple
                    'Republic of Ireland': '#118C7B',
                   'Holyhead': '#206095', # ocean blue
                   'Pembroke' :'#A8BD3A', # spring green
                   'Fishguard' : '#746CB1',
                   'Liverpool Birkenhead': '#118C7B'}

In [ ]:
def calc_index_yymm(df, group, need_datetime):
    
    if need_datetime is True:
        df['year'] = df['date_time'].dt.year
        df['month'] = df['date_time'].dt.month
    
    month_group = group + ['month']
    
    df = df.sort_values(['date_time']).reset_index(drop=True)
    # index spend
    df["index_spend"] = df.groupby(group)["spend"].transform(
        lambda x: x / x.iloc[0]
    )
    # index cardholders
    df["index_cardholders"] = df.groupby(group)["cardholders"].transform(
        lambda x: x / x.iloc[0]
    )
    # month-on-month
    df['mm_perc_spend'] = df.groupby(group)['index_spend'].pct_change(periods=1)*100
    df['mm_perc_cardholders'] = df.groupby(group)['index_cardholders'].pct_change(periods=1)*100

    # year-on-year
    df['yy_perc_spend'] = df.groupby(month_group)['index_spend'].pct_change(periods=1)*100
    df['yy_perc_cardholders'] = df.groupby(month_group)['index_cardholders'].pct_change(periods=1)*100
    
    return df

In [ ]:
def plot_charts(y, df):
    
    # Chart annotations
    if df.name == 'int_df':
        string = ''
        legend_string = ''
        merch_card = 'merchant_location'
    elif df.name == 'irish':
        legend_string = 'Cardholder Origin '
        merch_card = 'cardholder_location'
        
    elif df.name == 'df':
        string = ' domestic and'
        legend_string = ''
        merch_card = 'merchant_location'
        
    else: # specific port district'
        legend_string = 'Cardholder Origin '
        merch_card = 'cardholder_location'
        postal_district = df.name
        
    # Plotting metrics
    if df.name == 'df' or df.name == 'int_df': # df or int
        if y == 'index_spend':
            title=f"Indexed sum spend by{string} international cardholders at areas"
            title_subtitle_text='Indexed to January 2019' 
            yaxis_title='Indexed Spend'
            legend_title_text='Merchant location:'

        if y == 'index_cardholders':
            title=f"Indexed number of{string} international cardholders transacting at areas"
            title_subtitle_text='Indexed to January 2019' 
            yaxis_title='Indexed Number of Cardholders'
            legend_title_text='Merchant location:'

    elif df.name == 'irish':
        if y == 'index_spend':
            title=f"Indexed sum spend at merchants in LL65 by origin of cardholder"
            title_subtitle_text='Indexed to October 2022' 
            yaxis_title='Indexed Spend'
            legend_title_text='Cardholder origin:'

        if y == 'index_cardholders':
            title=f"Indexed sum cardholders transacting at merchants in LL65 by cardholder origin"
            title_subtitle_text='Indexed to October 2022' 
            yaxis_title='Indexed Number of Cardholders'
            legend_title_text='Cardholder origin:'
    else: # specific distirct
        if y == 'index_spend':
            title=f"Indexed Irish and Northern Irish cardholder sum spend at merchants in {postal_district}"
            title_subtitle_text='Indexed to October 2022' 
            yaxis_title='Indexed Spend'
            legend_title_text='Cardholder origin:'

        if y == 'index_cardholders':
            title=f"Indexed Irish and Northern Irish sum cardholders transacting at merchants in {postal_district}"
            title_subtitle_text='Indexed to October 2022' 
            yaxis_title='Indexed Number of Cardholders'
            legend_title_text='Cardholder origin:'
    
    if y == 'mm_perc_spend':
        title=f"Month-on-month % change in spend"
        title_subtitle_text=' ' 
        yaxis_title=' '
        legend_title_text=f'{legend_string}Location:'

    if y == 'mm_perc_cardholders':
        title=f"Month-on-month % change in cardholders"
        title_subtitle_text=' ' 
        yaxis_title=' '
        legend_title_text=f'{legend_string}Location:'

    if y == 'yy_perc_spend':
        title=f"Year-on-year % change in spend"
        title_subtitle_text=' ' 
        yaxis_title=' '
        legend_title_text=f'{legend_string}Location:'

    if y == 'yy_perc_cardholders':
        title=f"Year-on-year % change in cardholders"
        title_subtitle_text=' ' 
        yaxis_title=' '
        legend_title_text=f'{legend_string}Location:'
    
    fig = px.line(
    df,
    x="date_time",
    y=y,
    color = merch_card,
    title=title,
    template='simple_white',
    color_discrete_map=holyhead_colours, 
    height = 500,
    width = 900
        
    )
    fig.update_layout(title_subtitle_text=title_subtitle_text, yaxis =dict(title=yaxis_title),
                     legend_title_text=legend_title_text)
    
#     if df.name == 'df' or df.name =='irish':
#         fig.add_trace(
#         go.Scatter(
#             x=ship_df['date_time'],
#             y=ship_df['ships'],
#             mode='lines',
#             name='Holyhead passenger ship visits',
#             yaxis='y2',
#             line=dict(color='slategrey', dash='dash', width=1)
#         )
#         )

#         fig.update_layout(
#         yaxis=dict(title='Indexed spend'),
#         yaxis2=dict(
#             title='Holyhead passenger ship visits',
#             overlaying='y',
#             side='right',
#         )
#         )

#     else:
#         pass

 
    for start_date, end_date in [(f"2024-11-20", f"2024-12-01")]:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor='lightgrey',
            opacity=0.5,
            layer="below",
            line_width=0,
        )

         # for in-graph labelling: extract x and y for spend
    #         for location in all_dfs['merchant_location'].unique():
    #             spend_value = all_dfs[(all_dfs['merchant_location'] == location) & (all_dfs['date_time'] == f'202{n}-12-01')]['index_spend'].iloc[0]

    #             fig.add_annotation(
    #                 x=f"202{n}-12-1",
    #                 y=spend_value,  
    #                 text=f'{location} : {round(spend_value,2)}',
    #                 font=dict(size=8, color="black",style="italic"),  
    #                 align='center', 
    #             )

    # if df.name == 'irish':
    #     fig.update_traces(selector=dict(name='Rest of UK'), line=dict(dash='dot'))
    #     fig.update_traces(selector=dict(name='Rest of International'), line=dict(dash='dot'))
        

    if df.name != 'irish':
        fig.update_traces(selector=dict(name='Rest of UK'), line=dict(width = 1.5))
        fig.update_traces(selector=dict(name='Wider Anglesey'), line=dict(width = 1.5))
        fig.update_traces(selector=dict(name='LL65'), line=dict(width = 2.7))

    fig.update_layout(legend=dict(
        font = dict(size=12),
        orientation = 'h',
        xanchor = 'right',
        x=1,
        y=1.05,
        yanchor = 'top'
    ))
    fig.show()

#### High-level overview

In [ ]:
client = bigquery.Client()

sql = f"""SELECT time_period_value, merchant_location, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'POSTAL_DISTRICT' AND
  cardholder_issuing_level = 'All' AND
  mcg = 'All'
  ORDER BY time_period_value, merchant_location
  """

hp_full = client.query(sql).to_dataframe()
hp_full = t.create_date_time(hp_full)

# removing non-district locations 
hp_full["number"] = hp_full["merchant_location"].str.extract(
    "(\d+)", expand=False
)
hp_full = hp_full.loc[hp_full["number"].isna() == False].drop(
    "number", axis=1
)

ll65 = hp_full[hp_full['merchant_location'] == 'LL65'].copy()
area = hp_full[(hp_full['merchant_location'].isin(holyhead_districts)) & (hp_full['merchant_location'] != 'LL65')].copy()
uk = hp_full[(~hp_full['merchant_location'].isin(holyhead_districts)) & (hp_full['merchant_location'] != 'LL65')].copy()

area = (
        area.groupby(["date_time"])
        .agg({"spend": "sum", "transactions": "sum", "cardholders": "sum"})
        .reset_index()
    )
uk = (
        uk.groupby(["date_time"])
        .agg({"spend": "sum", "transactions": "sum", "cardholders": "sum"})
        .reset_index()
    )

area['merchant_location'] = 'Wider Anglesey'
uk['merchant_location'] = 'Rest of UK'

area = calc_index_yymm(area, group = ['merchant_location'], need_datetime = True)
uk = calc_index_yymm(uk, group = ['merchant_location'], need_datetime = True)
ll65 = calc_index_yymm(ll65, group = ['merchant_location'], need_datetime = True)

all_dfs = pd.concat([ll65, uk, area])
all_dfs.name = 'df'

all_dfs.columns[8:]

In [ ]:
plot_charts(y = 'index_spend', df = all_dfs)

In [ ]:
client = bigquery.Client()

sql = f"""SELECT time_period_value, merchant_location, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'POSTAL_DISTRICT' AND
  cardholder_issuing_level = 'International' AND
  cardholder_issuing_country = 'All' AND
  mcg = 'All' AND
  time_period_value >= '202111'
  ORDER BY time_period_value, merchant_location
  """

df_full_int = client.query(sql).to_dataframe()
df_full_int = t.create_date_time(df_full_int)

In [ ]:
# removing non-district locations 
df_full_int["number"] = df_full_int["merchant_location"].str.extract(
    "(\d+)", expand=False
)
df_full_int = df_full_int.loc[df_full_int["number"].isna() == False].drop(
    "number", axis=1
)

ll65 = df_full_int[df_full_int['merchant_location'] == 'LL65'].copy()
area = df_full_int[(df_full_int['merchant_location'].isin(holyhead_districts)) & (df_full_int['merchant_location'] != 'LL65')].copy()
uk = df_full_int[(~df_full_int['merchant_location'].isin(holyhead_districts)) & (df_full_int['merchant_location'] != 'LL65')].copy()

area = (
        area.groupby(["date_time"])
        .agg({"spend": "sum", "transactions": "sum", "cardholders": "sum"})
        .reset_index()
    )
uk = (
        uk.groupby(["date_time"])
        .agg({"spend": "sum", "transactions": "sum", "cardholders": "sum"})
        .reset_index()
    )

area['merchant_location'] = 'Wider Anglesey'
uk['merchant_location'] = 'Rest of UK'

area = calc_index_yymm(area, group = ['merchant_location'], need_datetime = True)
uk = calc_index_yymm(uk, group = ['merchant_location'], need_datetime = True)
ll65 = calc_index_yymm(ll65, group = ['merchant_location'], need_datetime = True)

all_dfs_int = pd.concat([ll65, uk, area])

all_dfs_int.name = 'int_df'

all_dfs_int.columns[8:]

In [ ]:
plot_charts(y = 'yy_perc_spend', df = all_dfs_int)

In [ ]:
plot_charts(y = 'yy_perc_cardholders', df = all_dfs_int)

In [ ]:
client = bigquery.Client()

sql = f"""SELECT *
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location = 'LL65' AND
  cardholder_issuing_level != 'All' AND
  cardholder_issuing_country IN ('All', 'UNITED KINGDOM') AND
  mcg = 'All' AND
  time_period_value >= '202111'
  ORDER BY time_period_value, merchant_location
  """

df_rat = client.query(sql).to_dataframe()
df_rat = t.create_date_time(df_rat)

df_rat["perc_spend"] = df_rat.groupby(["time_period_value"])['spend'].transform(lambda x: x / x.sum()*100)

In [ ]:
fig = px.line(
df_rat,
x="date_time",
y="perc_spend",
    color = 'cardholder_issuing_country',
template='simple_white',
height = 500,
width = 800)
for n in [1,2,3,4]:
        if n == 4:
            colour = 'lightgrey'
        else: 
            colour = 'gainsboro'
        for start_date, end_date in [(f"202{n}-11-20", f"202{n}-12-01")]:
            fig.add_shape(
                type="rect",
                xref="x",
                yref="paper",
                x0=start_date,
                y0=0,
                x1=end_date,
                y1=1,
                fillcolor=colour,
                opacity=0.5,
                layer="below",
                line_width=0,
            )

fig.show()

In [ ]:
df_rat[(df_rat['time_period_value'].str.contains('2023')) & (df_rat['cardholder_issuing_level'] == 'Domestic')]['perc_spend'].mean()

In [ ]:
# dom spend proportion 2023 average
(df_rat[(df_rat['time_period_value'].str.contains('2023')) & (df_rat['cardholder_issuing_level'] == 'Domestic')]['spend'].sum())/df_rat[(df_rat['time_period_value'].str.contains('2023'))]['spend'].sum()

In [ ]:
# december international 

df_rat['month'] = df_rat['date_time'].dt.month

(df_rat[(df_rat['month'] == 12) & (df_rat['year'] != 2024) & (df_rat['cardholder_issuing_level'] == 'International')])['spend'].sum()/(df_rat[(df_rat['month'] == 12) & (df_rat['year'] != 2024)])['spend'].sum()

In [ ]:
df_rat[df_rat['cardholder_issuing_level'] == 'International'].sort_values(by='perc_spend', ascending = False)[['perc_spend', 'date_time']]

In [ ]:
client = bigquery.Client()

sql = f"""SELECT *
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'POSTAL_DISTRICT' AND
  cardholder_issuing_level = 'International' AND
  cardholder_issuing_country = 'All' AND
  merchant_location = 'LL65' AND
  mcc = 'All' AND
  time_period_value >= '202111'
  ORDER BY time_period_value, merchant_location
  """

df_int_mcg = client.query(sql).to_dataframe()
df_int_mcg = t.create_date_time(df_int_mcg)
df_int_mcg = calc_index_yymm(df_int_mcg, group = ['mcg'], need_datetime = True)


In [ ]:
fig = px.line(
df_int_mcg,
x="date_time",
y="mm_perc_spend",
color = 'mcg',
template='simple_white',
height = 500,
width = 800)

fig.show()

#### Northern Irish and Republic of Ireland spend

In [ ]:
client = bigquery.Client()

sql_ire = f"""SELECT time_period_value, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location = 'LL65' AND
  cardholder_issuing_country = 'REPUBLIC OF IRELAND' AND
  mcg = 'All' AND
  time_period_value >= '202210'
  ORDER BY time_period_value
  """

df_ire = client.query(sql_ire).to_dataframe()
df_ire = t.create_date_time(df_ire)
df_ire['cardholder_location'] = 'REPUBLIC OF IRELAND'

sql_bt = f"""SELECT time_period_value, cardholder_location, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.retail_performance_high_streets_towns
  WHERE time_period = 'Month' AND 
  merchant_location = 'LL65' AND
  cardholder_location = 'BT' AND
  mcg = 'All' AND
  time_period_value >= '202201'
  ORDER BY time_period_value, cardholder_location
  """

df_bt = client.query(sql_bt).to_dataframe()
df_bt = t.create_date_time(df_bt)

# would need rest of UK and rest of international sums
sql_int = f"""SELECT time_period_value, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location = 'LL65' AND
  cardholder_issuing_level = 'International' AND
  cardholder_issuing_country != 'REPUBLIC OF IRELAND' AND
  mcg = 'All' AND
  time_period_value >= '202210'
  ORDER BY time_period_value
  """

df_int = client.query(sql_int).to_dataframe()
df_int = t.create_date_time(df_int)
df_int = (
        df_int.groupby(["date_time"])
        .agg({"spend": "sum", "transactions": "sum", "cardholders": "sum"})
        .reset_index()
    )
df_int['cardholder_location'] = 'Rest of International'

sql_uk = f"""SELECT time_period_value, cardholder_location, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.retail_performance_high_streets_towns
  WHERE time_period = 'Month' AND 
  merchant_location = 'LL65' AND
  cardholder_location != 'BT' AND
  cardholder_location != 'All' AND
  mcg = 'All' AND
  time_period_value >= '202210'
  ORDER BY time_period_value, cardholder_location
  """

df_uk = client.query(sql_uk).to_dataframe()
df_uk = t.create_date_time(df_uk)
df_uk = (
        df_uk.groupby(["date_time"])
        .agg({"spend": "sum", "transactions": "sum", "cardholders": "sum"})
        .reset_index()
    )
df_uk['cardholder_location'] = 'Rest of UK'

irish_df = pd.concat([df_bt, df_ire, df_uk, df_int]).reset_index(drop = True)
irish_df = calc_index_yymm(irish_df, group = ['cardholder_location'], need_datetime = True)

irish_df.name = 'irish'
irish_df.columns[8:]

irish_df['cardholder_location'] =irish_df['cardholder_location'].replace({'BT': 'Northern Ireland', 'REPUBLIC OF IRELAND' : 'Republic of Ireland'})


In [ ]:
plot_charts(y = 'index_spend', df = irish_df)

In [ ]:
client = bigquery.Client()

sql = f"""SELECT time_period_value, merchant_location, mcg, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'POSTAL_DISTRICT' AND
  cardholder_issuing_level = 'All' AND
  merchant_location = 'LL65' AND
    mcc = 'All' AND
  time_period_value >= '202111'
  ORDER BY time_period_value, merchant_location
  """

mcg_full = client.query(sql).to_dataframe()
mcg_full = t.create_date_time(mcg_full)


In [ ]:
# adjust spend for number of cardholders in Jan 2019
mcg_full["idx_cards"] = mcg_full.groupby(
        ["mcg"]
    )["cardholders"].transform(lambda x: x / x.iloc[0])

# spend / inded 
mcg_full["idx_spend"] = mcg_full["spend"] / mcg_full["idx_cards"]

group_list = [ 'mcg']

metrics = ['idx_spend', 'spend', 'transactions', 'cardholders']
month_group = group_list + ['month']

result = mcg_full.copy()
for i in metrics:
    # calc year-on-year differences
    result[f'yoy_{i}'] = result.groupby(month_group)[f'{i}'].diff(periods=1)

    # calc year-on-year % change
    result[f'yoy_{i}_perc'] = result.groupby(month_group)[f'{i}'].pct_change(periods=1)*100


    # index to jan 2019
    result[f"index_{i}"] = result.groupby(group_list)[f"{i}"].transform(
    lambda x: x / x.iloc[0]
)
  

In [ ]:
mcgs_interest = ['All', 'RESTAURANTS', 'FOOD & GROCERY', 'TRANSPORTATION']

In [ ]:
fig = px.line(
result[result['mcg'].isin(mcgs_interest)],
x="date_time",
y="index_spend",
color = 'mcg',
template='simple_white',
title = 'MCG spend at LL65 merchants',
height = 500,
width = 800)
for n in [1,2,3,4]:
        if n == 4:
            colour = 'lightgrey'
        else: 
            colour = 'gainsboro'
        for start_date, end_date in [(f"202{n}-11-20", f"202{n}-12-01")]:
            fig.add_shape(
                type="rect",
                xref="x",
                yref="paper",
                x0=start_date,
                y0=0,
                x1=end_date,
                y1=1,
                fillcolor=colour,
                opacity=0.5,
                layer="below",
                line_width=0,
            )

fig.show()

In [ ]:
fig = px.line(
result[result['mcg'].isin(mcgs_interest)],
x="date_time",
y="yoy_spend_perc",
color = 'mcg',
template='simple_white',
title = 'MCG spend at LL65 merchants',
height = 500,
width = 800)

fig.show()

#### Other ports

In [ ]:
def plot_other_ports(df, metric):
    fig = px.line(
    df,
    x="date_time",
    y=metric,
    color = 'merchant_location',
        color_discrete_map=port_colours, 
    title=f'Irish cardholder {metric} at port districts',
    template='simple_white',
    height = 500,
    width = 800)



    for n in [1,2,3,4]:
        if n == 4:
            colour = 'lightgrey'
        else: 
            colour = 'gainsboro'
        for start_date, end_date in [(f"202{n}-11-20", f"202{n}-12-01")]:
            fig.add_shape(
                type="rect",
                xref="x",
                yref="paper",
                x0=start_date,
                y0=0,
                x1=end_date,
                y1=1,
                fillcolor=colour,
                opacity=0.5,
                layer="below",
                line_width=0,
            )


    fig.update_traces(selector=dict(name='Holyhead'), line=dict(width = 3))
    for i in df_near['merchant_location'].unique():
        if i != 'Holyhead':
            fig.update_traces(selector=dict(name=i), opacity = .7)

    fig.update_layout(title_subtitle_text='Indexed to Nov 2021', 
                      #yaxis =dict(title='Indexed spend'),
                         #legend_title_text='Merchant location'
                     )
    fig.show()

In [ ]:
other_ports = ('CH41', 'SA64',  'LL65')
port_colours = {'Holyhead': '#A8BD3A',
                   'Fishguard' : '#746CB1',
                   'Birkenhead': '#F66068'}

client = bigquery.Client()

sql_nearby = f"""SELECT time_period_value, merchant_location, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
  WHERE time_period = 'Month' AND 
  merchant_location_level = 'POSTAL_DISTRICT' AND
  merchant_location IN {other_ports} AND
  cardholder_issuing_country = 'REPUBLIC OF IRELAND' AND
  mcg = 'All' AND
  time_period_value >= '202111'
  ORDER BY time_period_value, merchant_location
  """

df_near = client.query(sql_nearby).to_dataframe()
df_near = t.create_date_time(df_near)

df_near = calc_index_yymm(df_near, group = ['merchant_location'], need_datetime = True)


df_near['merchant_location'] =df_near['merchant_location'].replace({'CH41': 'Birkenhead', 
                                                                    'LL65' : 'Holyhead',
                                                                   'SA64' : 'Fishguard'})

In [ ]:
## High-level

# other_ports = ('CH41', 'SA64',  'LL65')
# port_colours = {'Holyhead': '#A8BD3A',
#                    'Fishguard' : '#746CB1',
#                    'Birkenhead': '#F66068'}

# client = bigquery.Client()

# sql_nearby = f"""SELECT *
#   FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
#   WHERE time_period = 'Month' AND 
#   merchant_location_level = 'POSTAL_DISTRICT' AND
#   merchant_location IN {other_ports} AND
#   cardholder_issuing_country = 'All' AND
#   cardholder_issuing_level = 'All' AND
#   mcg = 'All' AND
#   time_period_value >= '202111'
#   ORDER BY time_period_value, merchant_location
#   """

# df_near = client.query(sql_nearby).to_dataframe()
# df_near = t.create_date_time(df_near)

# df_near = calc_index_yymm(df_near, group = ['merchant_location'], need_datetime = True)


# df_near['merchant_location'] =df_near['merchant_location'].replace({'CH41': 'Birkenhead', 
#                                                                     'LL65' : 'Holyhead',
#                                                                    'SA64' : 'Fishguard'})

In [ ]:
fig = px.line(
df_near,
x="date_time",
y="index_spend",
color = 'merchant_location',
    color_discrete_map=port_colours, 
title=f'Irish cardholder indexed spend at port districts',
template='simple_white',
height = 500,
width = 800)



for n in [1,2,3,4]:
    if n == 4:
        colour = 'lightgrey'
    else: 
        colour = 'gainsboro'
    for start_date, end_date in [(f"202{n}-11-20", f"202{n}-12-01")]:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=colour,
            opacity=0.5,
            layer="below",
            line_width=0,
        )


fig.update_traces(selector=dict(name='Holyhead'), line=dict(width = 3))
for i in df_near['merchant_location'].unique():
    if i != 'Holyhead':
        fig.update_traces(selector=dict(name=i), opacity = .7)

fig.update_layout(title_subtitle_text='Indexed to Nov 2021', 
                  yaxis =dict(title='Indexed spend'),
                     legend_title_text='Merchant location'
                 )

fig.update_layout(legend=dict(
        font = dict(size=12),
        orientation = 'h',
        xanchor = 'right',
        x=1,
        y=1.05,
        yanchor = 'top'
    ))
fig.show()

'index_spend',
       'index_cardholders', 'mm_perc_spend', 'mm_perc_cardholders',
       'yy_perc_spend', 'yy_perc_cardholders'

In [ ]:
plot_other_ports(df_near, metric = 'mm_perc_spend')